In [71]:
import os 
import pandas as pd
from bs4 import BeautifulSoup

In [74]:
os.chdir("/Users/tunahankilic/Desktop/NBAGamePrediction")

In [75]:
os.getcwd()

'/Users/tunahankilic/Desktop/NBAGamePrediction'

In [76]:
BOXSCORES_DIR = "data/boxscores"

In [77]:
box_scores = os.listdir(BOXSCORES_DIR)

In [82]:
box_scores_filtered = [file for file in box_scores if any(year in file[:8] for year in ['2018', '2019', '2020', '2021', '2022', '2023'])]

In [83]:
box_score_path = [os.path.join(BOXSCORES_DIR, f) for f in box_scores_filtered if f.endswith(".html")]

In [85]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [86]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id":"line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = 'team'
    cols[-1] = 'total'
    line_score.columns = cols
    line_score = line_score[['team', 'total']]
    return line_score

In [87]:
def read_four_factors(soup):
    four_factors = pd.read_html(str(soup), attrs={"id":"four_factors"})[0]
    cols = list(four_factors.columns)
    cols[0] = 'team'
    four_factors.columns = cols
    four_factors = four_factors[['team', 'Pace', 'FT/FGA']]
    return four_factors

In [88]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors='coerce')
    return df

In [89]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [90]:
base_cols = None
games = []

#box_score = box_score_path[0]
for box_score in box_score_path:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    four_factors = read_four_factors(soup)
    teams = list(line_score['team'])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, 'basic')
        advanced = read_stats(soup, team, 'advanced')

        totals = pd.concat([basic.iloc[-1, :], advanced.iloc[-1, :]])
        team_four_factor = four_factors.loc[four_factors['team'] == team]
        summary = pd.concat([totals, team_four_factor.iloc[-1,1:]])
        summary.index = summary.index.str.lower()

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep='first'))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]

        summaries.append(summary)

    df_summary = pd.concat(summaries, axis=1).T

    game = pd.concat([df_summary, line_score], axis =1)
    game["home"] = [0, 1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)

    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_score_path)}")

df_games = pd.concat(games, ignore_index=True)

100 / 6349
200 / 6349
300 / 6349
400 / 6349
500 / 6349
600 / 6349
700 / 6349
800 / 6349
900 / 6349
1000 / 6349
1100 / 6349
1200 / 6349
1300 / 6349
1400 / 6349
1500 / 6349
1600 / 6349
1700 / 6349
1800 / 6349
1900 / 6349
2000 / 6349
2100 / 6349
2200 / 6349
2300 / 6349
2400 / 6349
2500 / 6349
2600 / 6349
2700 / 6349
2800 / 6349
2900 / 6349
3000 / 6349
3100 / 6349
3200 / 6349
3300 / 6349
3400 / 6349
3500 / 6349
3600 / 6349
3700 / 6349
3800 / 6349
3900 / 6349
4000 / 6349
4100 / 6349
4200 / 6349
4300 / 6349
4400 / 6349
4500 / 6349
4600 / 6349
4700 / 6349
4800 / 6349
4900 / 6349
5000 / 6349
5100 / 6349
5200 / 6349
5300 / 6349
5400 / 6349
5500 / 6349
5600 / 6349
5700 / 6349
5800 / 6349
5900 / 6349
6000 / 6349
6100 / 6349
6200 / 6349
6300 / 6349


In [101]:
df_games = df_games.loc[:,~df_games.columns.duplicated()]

In [104]:
df_games.to_parquet('./data/nba_games.parquet', engine='pyarrow')